In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
train = pd.read_csv("./data/final/train.csv")
dev = pd.read_csv("./data/final/dev.csv")

In [ ]:
allocine_infos = pd.read_csv("./data/externe/allocine_movies.csv", sep=',')

temp = train.merge(right=allocine_infos, right_on='id', left_on='movie', how='left')
temp_train = temp.drop(['id','press_rating','number_of_press_rating','spec_rating','number_of_spec_rating'], axis=1)
temp_train = temp_train.dropna()

temp_dev = dev.merge(right=allocine_infos, right_on='id', left_on='movie', how='left')
temp_dev = temp_dev.drop(['id','press_rating','number_of_press_rating','spec_rating','number_of_spec_rating'], axis=1)
temp_dev = temp_dev.dropna()

In [ ]:
cvec = CountVectorizer(
    analyzer = 'word',
    lowercase = False,
    ngram_range = (1,2)
)

preprocessor = ColumnTransformer(transformers=[
    ('tfidf1', cvec, "commentaire_clean"),
    ("scaler4", MinMaxScaler(), ["sentimental_score"])
])

In [ ]:
X_train = preprocessor.fit_transform(temp_train)
X_dev = preprocessor.transform(temp_dev)

In [ ]:
model = Sequential()
model.add(Dense(8, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
model.fit(X_train, temp_train["note"].values, epochs=30, verbose=1, validation_data=(X_dev, temp_dev["note"].values))